In [1]:
### Loading the Dataset

try: from datasets import load_dataset
except:
    !pip install datasets
    from datasets import load_dataset


# dataset = load_dataset("nvidia/OpenMathInstruct-2", split = "train_1M")
dataset = load_dataset("commonsense_qa")
# dataset_small = dataset.select(range(10000))
# dataset_split = dataset_small.train_test_split(test_size=0.1, seed=42)
# train_data, eval_data = dataset_split["train"], dataset_split["test"]
train_data, eval_data = dataset["train"], dataset["validation"]

print(train_data)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'question', 'question_concept', 'choices', 'answerKey'],
    num_rows: 9741
})


In [2]:
eval_data.to_pandas().head()

,id,question,question_concept,choices,answerKey
0,1afa02df02c908a558b4036e80242fac,A revolving door is convenient for two directi...,revolving door,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
1,a7ab086045575bb497933726e4e6ad28,What do people aim to do at work?,people,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
2,b8c0a4703079cf661d7261a60a1bcbff,Where would you find magazines along side many...,magazines,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",B
3,e68fb2448fd74e402aae9982aa76e527,Where are you likely to find a hamburger?,hamburger,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A
4,2435de612dd69f2012b9e40d6af4ce38,James was looking for a good place to buy farm...,farmland,"{'label': ['A', 'B', 'C', 'D', 'E'], 'text': [...",A


In [3]:
### Logging into HuggingFace

try:
  from dotenv import load_dotenv
except:
  !pip install python-dotenv
  from dotenv import load_dotenv

from huggingface_hub import login, whoami
import os

load_dotenv()
login(os.getenv("hugging_face_key2"))

In [4]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 103.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, PrefixTuningConfig, TaskType


model_name = "gpt2"
# model_name = "tiiuae/falcon-7b"
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

peft_config = PrefixTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    num_virtual_tokens=10,        # prefix length
    prefix_projection=True,       # optional MLP projection of prefix
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

trainable params: 14,772,480 || all params: 139,212,288 || trainable%: 10.6115


In [6]:
# def format_prompt(example):
#     text = f"[INST] Problem: {example['problem']} [/INST] Solution: {example['generated_solution']}"
#     tokenized = tokenizer(
#         text,
#         padding="max_length",
#         truncation=True,
#         max_length=512
#     )
#     tokenized["labels"] = tokenized["input_ids"].copy()
#     return tokenized

# tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
# tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

# tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
# tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])



import numpy as np
def format_prompt(example):
    question = example['question']
    choices = example['choices']['text']
    labels = example['choices']['label']
    answer_key = example['answerKey']

    # Find the correct answer text
    answer_index = labels.index(answer_key)
    answer_text = choices[answer_index]

    # Create the prompt
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    full_text = prompt + " " + answer_text

    # Tokenize full input
    tokenized = tokenizer(
        full_text,
        padding="max_length",
        truncation=True,
        max_length=128,
    )

    # Calculate where the completion starts
    prompt_ids = tokenizer(prompt,
                           truncation=True,
                           max_length=128)["input_ids"]
    completion_start = len(prompt_ids)

    # Mask prompt tokens in the labels
    labels = [-np.inf] * completion_start + tokenized["input_ids"][completion_start:]
    labels += [-np.inf] * (128 - len(labels))  # pad label to match input

    tokenized["labels"] = labels
    return tokenized


tokenized_train = train_data.map(format_prompt, batched=False, remove_columns=train_data.column_names)
tokenized_eval = eval_data.map(format_prompt, batched=False, remove_columns=eval_data.column_names)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/9741 [00:00<?, ? examples/s]

Map:   0%|          | 0/1221 [00:00<?, ? examples/s]

In [8]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


training_args = TrainingArguments(
    output_dir=f"./prefix_{model_name}",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="steps",
    eval_steps=100,
    save_steps=1000,
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",

)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    data_collator=data_collator,
)


trainer.train()



No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
100,3.865200,3.136809
200,3.128600,2.973499
300,2.979200,2.909287
400,2.959100,2.892230
500,2.875500,2.833773
600,2.858700,2.806686
700,2.854300,2.788202
800,2.822500,2.769948
900,2.797900,2.755414
1000,2.803100,2.748092


TrainOutput(global_step=3654, training_loss=2.7730573248067185, metrics={'train_runtime': 301.4333, 'train_samples_per_second': 96.947, 'train_steps_per_second': 12.122, 'total_flos': 1908934262784000.0, 'train_loss': 2.7730573248067185, 'epoch': 3.0})

In [12]:
peft_model.save_pretrained(f"./prefix_{model_name}_final")
tokenizer.save_pretrained(f"./prefix_{model_name}_final")

('./prefix_gpt2_final/tokenizer_config.json',
 './prefix_gpt2_final/special_tokens_map.json',
 './prefix_gpt2_final/vocab.json',
 './prefix_gpt2_final/merges.txt',
 './prefix_gpt2_final/added_tokens.json',
 './prefix_gpt2_final/tokenizer.json')

In [13]:
from peft import PeftModel, PeftConfig
peft_config = PeftConfig.from_pretrained(f"./prefix_{model_name}_final")

base_model = AutoModelForCausalLM.from_pretrained(peft_config.base_model_name_or_path)
model = PeftModel.from_pretrained(base_model, f"./prefix_{model_name}_final")

tokenizer = AutoTokenizer.from_pretrained(f"./prefix_{model_name}_final")

In [14]:
from transformers import pipeline
import torch
from tqdm import tqdm

# Use the PEFT model (already trained) and tokenizer
peft_model.eval()
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

def format_prompt(example):
    question = example["question"]
    choices = example["choices"]["text"]
    prompt = f"Q: {question}\nO: {' /// '.join(choices)}\nA: "
    return prompt, choices

correct = 0
total = 0

for example in tqdm(eval_data, total=len(eval_data)):
    prompt, choices = format_prompt(example)

    # Generate prediction
    output = pipe(prompt, max_new_tokens=10, do_sample=False, eos_token_id=tokenizer.eos_token_id)[0]["generated_text"]

    # Extract generated answer (after "Answer:")
    generated = output.split("\n")[2].split("A:")[-1].strip().split(" /// ")[0].strip()


    # Get ground truth
    answer_key = example["answerKey"]
    labels = example["choices"]["label"]
    true_answer = example["choices"]["text"][labels.index(answer_key)].strip()

    print("\n".join(output.split("\n")[:3]))
    print(f"\n\nModel: {generated} and Actual Answer: {true_answer}")

    if generated == true_answer:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Accuracy: {accuracy:.2%}")


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

Q: A revolving door is convenient for two direction travel, but it also serves as a security measure at a what?
O: bank /// library /// department store /// mall /// new york
A:  department store /// mall /// new york


Model: department store and Actual Answer: bank
Q: What do people aim to do at work?
O: complete job /// learn from each other /// kill animals /// wear hats /// talk to each other
A:  complete job /// complete job /// kill animals


Model: complete job and Actual Answer: complete job


  0%|          | 4/1221 [00:00<02:11,  9.26it/s]

Q: Where would you find magazines along side many other printed works?
O: doctor /// bookstore /// market /// train station /// mortuary
A:  bookstore /// mortuary


Model: bookstore and Actual Answer: bookstore
Q: Where are  you likely to find a hamburger?
O: fast food restaurant /// pizza /// ground up dead cows /// mouth /// cow carcus
A:  ground up dead cows /// mouth


Model: ground up dead cows and Actual Answer: fast food restaurant


  0%|          | 6/1221 [00:00<02:11,  9.26it/s]

...

Q: What would you do to a rock when climb up a cliff?
O: grab /// look down /// throw /// falling /// may fall
A:  fall down /// fall down /// fall down


Model: fall down and Actual Answer: grab
Q: His compressor needed a new hose, where did he go?
O: garden shed /// hardware store /// brothel /// garage /// greenhouse
A:  garage /// brothel


Model: garage and Actual Answer: hardware store


100%|█████████▉| 1220/1221 [02:12<00:00,  9.29it/s]

Q: The man closed his eyes as the music played, what effect did the music have?
O: coma /// enjoyable /// soothing /// universal /// good or bad
A:  soothing /// soothing /// universal


Model: soothing and Actual Answer: soothing
Q: Setting up framing, truss and beam are some of the first steps in what?
O: new construction /// warehouse /// driving /// ceiling /// bridge
A:  new construction /// bridge


Model: new construction and Actual Answer: new construction


100%|██████████| 1221/1221 [02:12<00:00,  9.21it/s]

Q: What is another name for a disk for storing information?
O: computer store /// computer to store data /// computer hard drive /// cd player /// usb mouse
A:  computer store /// computer to store data


Model: computer store and Actual Answer: computer hard drive
Accuracy: 29.07%


In [15]:
model.eval()
import torch
# prompt = f"Q: {question}\nChoices:\n"
#     for choice in choices:
#         prompt += f"- {choice}\n"
#     prompt += "Answer:"
q = "What do people aim to do at work?"
options = ['complete job', 'learn from each other', 'kill animals', 'wear hats', 'talk to each other']
options2 = ' /// '.join(options)
from IPython.display import display, Math
prompt = f"Q: {q}\nO: {options2}\nA: "
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=10,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    early_stopping=True,
    num_beams=5,  # Optional: beam search for stronger outputs
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Q: What do people aim to do at work?
O: complete job /// learn from each other /// kill animals /// wear hats /// talk to each other
A:  complete job /// complete job /// kill animals
A
